 # LLM Fine-Tuning with Alpaca Dataset

*** Install Required Libraries.***

In [1]:
!pip install -q transformers datasets peft accelerate bitsandbytes trl

**Importing Libraries.**

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
from trl import SFTTrainer

**Load the Dataset (Alpaca).**

In [3]:
dataset = load_dataset("tatsu-lab/alpaca")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [4]:
print(dataset["train"][0])

{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}


# 4: Load Base Model & Tokenizer (FLAN-T5-Small)

In [5]:
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

#5: Preprocess Dataset

In [6]:
def preprocess(example):
    prompt = example["instruction"] + "\n" + example["input"]
    inputs = tokenizer(prompt, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    labels = tokenizer(example["output"], truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    return {
        "input_ids": inputs["input_ids"][0],
        "attention_mask": inputs["attention_mask"][0],
        "labels": labels["input_ids"][0]
    }

tokenized_dataset = dataset["train"].map(preprocess)

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

#6: Apply PEFT (LoRA) Configuration

In [7]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [8]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 344,064 || all params: 77,305,216 || trainable%: 0.4451


#Step 7: Training Arguments

In [9]:
training_args = TrainingArguments(
    output_dir="./flan-t5-lora-alpaca",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,  # Use mixed precision on T4
    report_to="none"
)

**Trainer.**

In [10]:
from trl import SFTTrainer

In [13]:
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args
)

Truncating train dataset:   0%|          | 0/52002 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


*** Start Training.***

In [24]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,0.000000
20,0.000000
30,0.000000
40,0.000000
50,0.000000
60,0.000000
70,0.000000
80,0.000000
90,0.000000
100,0.000000


TrainOutput(global_step=19503, training_loss=0.0, metrics={'train_runtime': 7873.5425, 'train_samples_per_second': 19.814, 'train_steps_per_second': 2.477, 'total_flos': 2.916492689748787e+16, 'train_loss': 0.0})

**Save Fine-Tuned Mode.**

In [14]:
trainer.model.save_pretrained("./flan-t5-lora-alpaca")
tokenizer.save_pretrained("./flan-t5-lora-alpaca")

('./flan-t5-lora-alpaca/tokenizer_config.json',
 './flan-t5-lora-alpaca/special_tokens_map.json',
 './flan-t5-lora-alpaca/spiece.model',
 './flan-t5-lora-alpaca/added_tokens.json',
 './flan-t5-lora-alpaca/tokenizer.json')

**Evaluation .**

In [15]:
test_prompts = [
    "Translate English to French:\n My name is Touseef Ahmed and I am doing LLM-Fine Tunning on Alpaca Data-Set.",
    "What is the capital of Pakistan?",
    "Summarize the following:\nLarge language models are transforming AI development.",
]

In [16]:
for prompt in test_prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output_ids = model.generate(**inputs, max_new_tokens=50)
    print(f"\nPrompt: {prompt}")
    print("Response:", tokenizer.decode(output_ids[0], skip_special_tokens=True))


Prompt: Translate English to French:
 My name is Touseef Ahmed and I am doing LLM-Fine Tunning on Alpaca Data-Set.
Response: M'ai nom établi Touseef Ahmed et je suis d'ai d'ai d'ai d'ai d'ai d'ai

Prompt: What is the capital of Pakistan?
Response: sabah

Prompt: Summarize the following:
Large language models are transforming AI development.
Response: The AI is transforming the way we use it.
